In [1]:
# softmax_zoo_multi_classification

# multi-classification
# multi-normal classification (다중 분류) : Y값의 범주가 3개 이상인 분류
# 

In [2]:
import tensorflow as tf
import numpy as np
tf.random.set_seed(42)

In [3]:
xy = np.loadtxt("data-04-zoo.csv", delimiter=',',dtype=np.float32)
xy.shape #(101,17)

#학습 데이터 분리 70%
x_train = xy[:70,:-1]
y_train = xy[:70,[-1]]
print(x_train.shape,y_train.shape)
#검증 데이터 분리 30%
x_test = xy[70:,:-1]
y_test = xy[70:,[-1]]
print(x_test.shape,y_test.shape)

(70, 16) (70, 1)
(31, 16) (31, 1)


In [4]:
# One-hot 인코딩
# 1 - [ 1, 0, 0, 0, 0, 0, 0]
# 1 - [ 0, 1, 0, 0, 0, 0, 0]
#.....
# 7 - [ 0, 0, 0, 0, 0, 0, 1]
nb_classes = 7
Y_one_hot = tf.one_hot(y_train, nb_classes)
print(Y_one_hot.shape)
Y_one_hot = tf.reshape(Y_one_hot,[-1,nb_classes])
print(Y_one_hot.shape)

(70, 1, 7)
(70, 7)


In [5]:
#변수 초기화
# (70,16) * (16,7) = (70,7)

W = tf.Variable(tf.random.normal([16, nb_classes]),name='weight')
b = tf.Variable(tf.random.normal([nb_classes]),name='bias')

In [20]:
# 예측 함수(hypothesis) : H(x) = softmax(W*X + b)
def logits(X):
    return tf.matmul(X,W) + b
    
def hypothesis(X):
    return tf.nn.softmax(logits(X))

In [21]:
# 비용함수 구현 방법 : tf.nn.softmax_cross_entropy_with_logits() 함수 사용
def cost_func():
    cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits(x_train),
                                                     labels = Y_one_hot)
    cost = tf.reduce_mean(cost_i)
    return cost

In [22]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

In [23]:
# 학습 시작
print('***** Start Learning!!')
for step in range(10000):
    with tf.GradientTape() as tape:
        cost = cost_func()
    gradients = tape.gradient(cost, [W, b])
    optimizer.apply_gradients(zip(gradients, [W, b]))
    
    if step % 100 == 0:
        print('%04d' % step, '손실 값: [', cost.numpy(), ']')
print('***** Learning Finished!!')

***** Start Learning!!
0000 손실 값: [ 1.07288336e-07 ]
0100 손실 값: [ 7.663452e-08 ]
0200 손실 값: [ 6.982256e-08 ]
0300 손실 값: [ 6.641659e-08 ]
0400 손실 값: [ 5.960463e-08 ]
0500 손실 값: [ 4.768371e-08 ]
0600 손실 값: [ 4.598072e-08 ]
0700 손실 값: [ 4.598072e-08 ]
0800 손실 값: [ 4.427773e-08 ]
0900 손실 값: [ 3.23568e-08 ]
1000 손실 값: [ 3.0653812e-08 ]
1100 손실 값: [ 2.8950824e-08 ]
1200 손실 값: [ 2.0435875e-08 ]
1300 손실 값: [ 1.5326906e-08 ]
1400 손실 값: [ 1.7029896e-08 ]
1500 손실 값: [ 1.5326906e-08 ]
1600 손실 값: [ 1.5326906e-08 ]
1700 손실 값: [ 1.7029896e-08 ]
1800 손실 값: [ 1.7029896e-08 ]
1900 손실 값: [ 1.7029896e-08 ]
2000 손실 값: [ 1.5326906e-08 ]
2100 손실 값: [ 1.5326906e-08 ]
2200 손실 값: [ 6.8119586e-09 ]
2300 손실 값: [ 6.8119586e-09 ]
2400 손실 값: [ 6.8119586e-09 ]
2500 손실 값: [ 6.8119586e-09 ]
2600 손실 값: [ 6.8119586e-09 ]
2700 손실 값: [ 6.8119586e-09 ]
2800 손실 값: [ 5.108969e-09 ]
2900 손실 값: [ 8.514948e-09 ]
3000 손실 값: [ 6.8119586e-09 ]
3100 손실 값: [ 5.108969e-09 ]
3200 손실 값: [ 6.8119586e-09 ]
3300 손실 값: [ 5.108969e-09 ]
3400

In [30]:
# 정확도 측정 : accuracy computation

# y_test 값의 one-hot 인코딩
Y_one_hot = tf.one_hot(y_test,nb_classes)   
print(Y_one_hot.shape)                       # [31,1,7]  , Rank=3 (3차원)
Y_one_hot = tf.reshape(Y_one_hot,[-1,nb_classes])
print(Y_one_hot.shape)                       # [31,7]  , Rank=2 (2차원)


# tf.argmax() : 값이 가장 큰 요소의 인덱스 값을 반환
def predict(X):
    return tf.argmax(hypothesis(X),axis=1)

correct_predict = tf.equal(predict(x_test),tf.argmax(Y_one_hot,1))
accuracy = tf.reduce_mean(tf.cast(correct_predict, dtype = tf.float32))
print("Accuracy:",accuracy.numpy())

(31, 1, 7)
(31, 7)
Accuracy: 0.83870965


In [32]:
tf.reduce_sum(tf.cast(correct_predict, dtype = tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=26.0>

In [33]:
#예측
pred = predict(x_test).numpy()
pred

array([0, 1, 5, 3, 0, 0, 2, 6, 1, 1, 2, 5, 3, 1, 0, 6, 3, 1, 5, 4, 1, 4,
       3, 0, 0, 1, 0, 5, 0, 5, 1], dtype=int64)

In [34]:
for p,y in zip(pred, y_test.flatten()):
    print("[{}] Prediction: {} / Real Y: {}".format(p == int(y), p, int(y)))

[True] Prediction: 0 / Real Y: 0
[True] Prediction: 1 / Real Y: 1
[False] Prediction: 5 / Real Y: 6
[True] Prediction: 3 / Real Y: 3
[True] Prediction: 0 / Real Y: 0
[True] Prediction: 0 / Real Y: 0
[True] Prediction: 2 / Real Y: 2
[True] Prediction: 6 / Real Y: 6
[True] Prediction: 1 / Real Y: 1
[True] Prediction: 1 / Real Y: 1
[True] Prediction: 2 / Real Y: 2
[False] Prediction: 5 / Real Y: 6
[True] Prediction: 3 / Real Y: 3
[True] Prediction: 1 / Real Y: 1
[True] Prediction: 0 / Real Y: 0
[True] Prediction: 6 / Real Y: 6
[True] Prediction: 3 / Real Y: 3
[True] Prediction: 1 / Real Y: 1
[True] Prediction: 5 / Real Y: 5
[True] Prediction: 4 / Real Y: 4
[False] Prediction: 1 / Real Y: 2
[False] Prediction: 4 / Real Y: 2
[True] Prediction: 3 / Real Y: 3
[True] Prediction: 0 / Real Y: 0
[True] Prediction: 0 / Real Y: 0
[True] Prediction: 1 / Real Y: 1
[True] Prediction: 0 / Real Y: 0
[True] Prediction: 5 / Real Y: 5
[True] Prediction: 0 / Real Y: 0
[False] Prediction: 5 / Real Y: 6
[True